In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="qN4kXsL6ciaQC2x0nvIA")
project = rf.workspace("minh-huy").project("mask-unmask-classifier")
dataset = project.version(2).download("folder")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Mask-UnMask-Classifier-2 in folder:: 100%|██████████| 8444/8444 [00:02<00:00, 3176.15it/s]


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GroupNormalization
import cv2
import random
import tensorflow as tf
import matplotlib.pyplot as plt

src_path_train = "/content/Mask-UnMask-Classifier-2/train"
src_path_test = "/content/Mask-UnMask-Classifier-2/valid"

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        )

test_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",)
batch_size = 8
train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    target_size=(100, 100),
    color_mode="rgb",
    batch_size=8,
    class_mode="categorical",
    shuffle=False,
    seed=42
)
import tqdm
import numpy as np
train_generator.reset()
X_train, y_train = next(train_generator)
for i in tqdm.tqdm(range(int(train_generator.n/batch_size)-1)):
    img, label = next(train_generator)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)
import tqdm
import numpy as np
test_generator.reset()
X_test, y_test = next(test_generator)
for i in tqdm.tqdm(range(int(test_generator.n/batch_size)-1)):
    img, label = next(test_generator)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)
print(X_test.shape, y_test.shape)


Found 5904 images belonging to 2 classes.
Found 1685 images belonging to 2 classes.


100%|██████████| 737/737 [02:09<00:00,  5.67it/s]


(5904, 100, 100, 3) (5904, 2)


100%|██████████| 209/209 [00:15<00:00, 13.91it/s]

(1680, 100, 100, 3) (1680, 2)


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Activation,Dense
from tensorflow.keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import joblib

# Create the CNN model
sec = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
])

# Extract features from the CNN
cnn_features = sec.predict(X_train)

rf = RandomForestClassifier(n_estimators=100, max_depth=None)
rf.fit(cnn_features, y_train)




cnn_features_test = sec.predict(X_test)
y_pred = rf.predict(cnn_features_test)

accuracy = accuracy_score(y_test, y_pred)
print("Best Accuracy:", accuracy)








53/53 [==============================] - 0s 5ms/step
Best Accuracy: 0.9857142857142858


In [4]:
import joblib

best_rf_model_filename = '/content/model.joblib'
joblib.dump(rf, best_rf_model_filename)

['/content/model.joblib']